# Main

Saving pandas dataframes to excel with the function `pd.ExcelWriter`, there are several argument choices for the parameter `engine` that exist:

**`openpyxl`**
- Default & append‑friendly  
- **Use when:** You need to **append** to existing `.xlsx` files.  
- **Pros:** 
  - Can modify existing workbooks with the append mode; 
  - widely supported.  
- **Cons:** 
  - Less advanced formatting than `xlsxwriter`.

**`xlsxwriter`**
- Best for rich, fresh reports that are highly-formatted  
- **Use when:** Creating **new** files with lots of formatting or charts.  
- **Pros:** 
  - Excellent styling, charts, and formatting tools.  
- **Cons:** 
  - Write‑only: cannot append to existing files


**`odf`**
- OpenDocument `.ods`  
- **Use when:** LibreOffice / open‑standard environments.  
- **Pros:** 
  - Vendor‑neutral.  
- **Cons:** 
  - Limited Excel‑style formatting.

**`xlwt`**
- Legacy `.xls`
- **Use when:** Forced to produce old `.xls` files.  
- **Pros:** 
  - Works with legacy Excel versions.  
- **Cons:** 
  - Very restrictive row/column limits; outdated.


# xlsxwriter

## generic excel write

In [6]:
import pandas as pd

def generic_excel_write(INPUT: str|pd.DataFrame, 
                        INPUT_DELIMITER: str = ',',
                        OUTPUT_RELATIVE_PATH: str = None):
    """
    # Parameters
    
    INPUT
        [str or pandas dataframe] input file that you want to process into excel. 
        If data type = str, then it's a local file that you have to read.
        If data type = pandas dataframe, then you passed in the pandas dataframe.
    
    INPUT_DELIMITER
        Delimiter that you want to use. Only applicable when you read a csv file as an input. 
    
    OUTPUT_RELATIVE_PATH
        Output name for your output file. 
    """
    # Read ifs
    if type(INPUT) == str:
        df = pd.read_csv(INPUT, 
                    sep=INPUT_DELIMITER
        )
        # display(df)
    elif type(INPUT) == pd.core.frame.DataFrame:
        df = INPUT.copy(deep=True)
    else:
        raise TypeError('unknown input format')
    # making output file name ifs
    if not OUTPUT_RELATIVE_PATH:
        if type(INPUT) == str:
            output_relative_path = '.'.join( INPUT.split('.')[:-1] ) + '.xlsx'
        else:
            output_relative_path = 'output.xlsx'
    else:
        output_relative_path = OUTPUT_RELATIVE_PATH
    #
    with pd.ExcelWriter(output_relative_path, engine='xlsxwriter') as writer:
        workbook = writer.book
        # tab to scroll item ratio
        workbook.set_tab_ratio(30)
        # write to main sheet
        df.to_excel(writer, 
                    sheet_name='Main',
                    index=False)
        worksheet1 = writer.sheets['Main']
        # auto-adjust column width
        for i, col in enumerate(df):
            width = min(
                max(
                    df[col].apply(lambda x: len(str(x))).max(),
                    len(col)),
                80) + 5
            worksheet1.set_column(i, i, width, workbook.add_format({'text_wrap': True,
                                                                    'valign': 'top'}))
        # put filter at the top of each column
        (max_row, max_col) = df.shape
        worksheet1.autofilter(0, 0, max_row, max_col-1)
        # format column headers
        columns = list(df.columns)
        for index, column_name in enumerate(columns):
            worksheet1.write(
                0, # row 0 
                index, # index of each column 
                column_name, # name of each column
                workbook.add_format({'bg_color': '#951F06',
                                    'valign': 'vcenter',
                                    'align': 'center',
                                    'bold': True,
                                    'font_color': '#FFFFFF'}))
        # Freeze the top row
        worksheet1.freeze_panes(1, 0)
    return None


# ### DEFINE YOUR VARIABLES HERE
# INPUT_RELATIVE_PATH = 'datasets/CASP14_2020.csv'
# INPUT_RELATIVE_PATH = 'datasets/emails.csv'
# INPUT_DELIMITER = '\t'
# generic_excel_write(INPUT_RELATIVE_PATH, 
#                     INPUT_DELIMITER)

df_test = pd.DataFrame({
    'Name':['John Wayne', 'Nicolas Cage', 'Homer Simpson'],
    'age': [100, 30, 20]})

for i, j in (
    ('my_datasets/CASP14_2020.csv', '\t'),
    ('my_datasets/emails.csv', ','),
    ('my_datasets/players.csv', ','),
    ('my_datasets/survey_results_public_trimmed-2.csv', ',')
):
    output_name = i.replace('my_datasets/', 'ignore_these/').replace('.csv', '.xlsx')
    print(i, output_name)
    generic_excel_write(
        i, 
        j,
        output_name
        )


my_datasets/CASP14_2020.csv ignore_these/CASP14_2020.xlsx
my_datasets/emails.csv ignore_these/emails.xlsx
my_datasets/players.csv ignore_these/players.xlsx
my_datasets/survey_results_public_trimmed-2.csv ignore_these/survey_results_public_trimmed-2.xlsx


## Ex2 (planets)

In [ ]:
import pandas as pd

df = pd.read_csv('datasets/planets-info/planets_data.csv')
df['solar_constant * magnetic_field_strength'] = 0

print(df.columns)
df.head(3)

Index(['planetOrder', 'name', 'description', 'wikiLink', 'imgSrc', 'type',
       'diameter_km', 'mass_kg', 'distance_from_sun_km',
       'average_orbital_distance_AU', 'orbital_period_days',
       'rotation_period_hours', 'number_of_moons', 'ring_system',
       'global_magnetic_field', 'average_temperature_celsius',
       'surface_pressure_bars', 'atmospheric_composition', 'notable_features',
       'escape_velocity_km_s', 'axial_tilt_degrees', 'albedo', 'gravity_m_s2',
       'day_length_earth_days', 'density_g_cm3', 'surface_composition',
       'volume_km3', 'mean_orbital_speed_km_s', 'orbital_eccentricity',
       'synodic_period_days', 'orbital_inclination_degrees',
       'surface_area_km2', 'solar_constant_W_m2', 'magnetic_field_strength_uT',
       'solar_constant * magnetic_field_strength'],
      dtype='object')


,planetOrder,name,description,wikiLink,imgSrc,type,diameter_km,mass_kg,distance_from_sun_km,average_orbital_distance_AU,...,surface_composition,volume_km3,mean_orbital_speed_km_s,orbital_eccentricity,synodic_period_days,orbital_inclination_degrees,surface_area_km2,solar_constant_W_m2,magnetic_field_strength_uT,solar_constant * magnetic_field_strength
0,1,Mercury,Mercury is the smallest planet in the Solar Sy...,https://en.wikipedia.org/wiki/Mercury_(planet),{'img': 'https://upload.wikimedia.org/wikipedi...,Terrestrial,4879,3.300000e+23,57900000.0,0.39,...,"Silicate rock, iron core",6.083000e+10,47.36,0.2056,116.75,7.00,74800000.0,6.3,0.3,0
1,2,Venus,Venus is the second planet from the Sun and is...,https://en.wikipedia.org/wiki/Venus,{'img': 'https://upload.wikimedia.org/wikipedi...,Terrestrial,12104,4.870000e+24,108200000.0,0.72,...,"Rocky, volcanic surface",9.284300e+11,35.02,0.0068,583.92,3.39,460000000.0,2610.0,0.0,0
2,3,Earth,Earth is the third planet from the Sun and the...,https://en.wikipedia.org/wiki/Earth,{'img': 'https://upload.wikimedia.org/wikipedi...,Terrestrial,12742,5.970000e+24,149600000.0,1.00,...,"Silicate crust, liquid water",1.083210e+12,29.78,0.0167,365.25,0.00,510000000.0,1361.0,25.0,0


In [ ]:
import pandas as pd
import seaborn as sns
from xlsxwriter.utility import xl_rowcol_to_cell

writer = pd.ExcelWriter('datasets/planets-info/planets_data_proc.xlsx', engine='xlsxwriter')
workbook = writer.book

df.to_excel(writer, 
            sheet_name='main', 
            index=False)
worksheet = writer.sheets['main']
for i, col in enumerate(df):
    if col == 'description':
        worksheet.set_column(i, i, 5, workbook.add_format({'bold': True}))
    if col == 'wikiLink':
        for row_num, url in enumerate(df['wikiLink'], start=1):
            worksheet.write_url(row_num, i, url, workbook.add_format({'font_color': 'green'}))

writer.close()

In [ ]:
df['wikiLink'].iloc[0]

'https://en.wikipedia.org/wiki/Mercury_(planet)'

In [ ]:
import re

for i in [
    'this is not a string!',
    'https://en.wikipedia.org/wiki/Mercury_(planet)'
]:
    output = re.findall(r'^https?://\S+', i)
    print(output)
    print(len(output))

[]
0
['https://en.wikipedia.org/wiki/Mercury_(planet)']
1


In [ ]:
import pandas as pd
import seaborn as sns
from xlsxwriter.utility import xl_rowcol_to_cell

writer = pd.ExcelWriter('datasets/planets-info/planets_data_proc.xlsx', engine='xlsxwriter')
### Access the workbook and worksheet objects
workbook = writer.book
# workbook writing
workbook.set_tab_ratio(30)


empty_df = pd.DataFrame()
empty_df.to_excel(writer, sheet_name='intro')
worksheet0 = writer.sheets['intro']
worksheet0.merge_range('A1:AS1', '      Title here', workbook.add_format({'font_size': 20,
                                                                          'font_color': '#333333'}))

### Write each dataframe slice to a different sheet
df.to_excel(writer, 
            sheet_name='main', 
            index=False)

def apply_format_to_sheet(worksheet, 
                          name_sheet: str, 
                          df: pd.DataFrame, 
                          chosen_format: dict[str:dict]):
    for i, col in enumerate(df):
        width = min(
            max(
                df[col].apply(lambda x: len(str(x))).max(),
                len(col)),
            80) + 5
        if col not in chosen_format:
            worksheet.set_column(i, i, width)
        else:
            worksheet.set_column(i, i, width, workbook.add_format(chosen_format[col]))
    return None

chosen_format = {
    'planetOrder': {'align':'center',
                    'valign': 'vcenter'},
    'name': {'align':'center',
             'valign': 'vcenter',
            #  'bg_color': '#951F06',
             'bold': True,
            #  'font_color': '#FFFFFF',
            #  'border': 1,
            #  'border_color': '#D3D3D3'
             },
    'description': {'text_wrap':True, 'font_size':20},
    'wikiLink': {'align': 'right', 'bold': True, 'font_size':20},
    'width': {'align':'center', 'num_format':'#,##0.00'},
    'diameter_km': {'align':'right', 'num_format':'#,##0'},
    'distance_from_sun_km': {'align':'right', 'num_format':'#,##0'}
}
worksheet1 = writer.sheets['main']
worksheet1.set_zoom(90)
apply_format_to_sheet(worksheet1,
                      'main',
                      df,
                      chosen_format)
# set colour to row 3 (Earth)
# worksheet1.set_row(3, None, 
                #    workbook.add_format({
                    #    'bg_color': '#ADD8E6'
                    # 'bold': True
                    #    }))

# conditional formatting
# worksheet.conditional_format('G2:G1000', {'type':'2_color_scale'})
worksheet1.conditional_format('G2:G10000', {'type': 'data_bar',
                                            'bar_solid': True})
worksheet1.conditional_format('I2:I10000', {'type': '2_color_scale',
                                            'min_color': 'white',
                                            'max_color': '#A91B0D'})
worksheet1.conditional_format('M2:M10000', {'type': '2_color_scale',
                                            'min_color': 'white',
                                            'max_color': '#ff6c74'})
worksheet1.conditional_format('R2:R10000', {'type': 'text',
                                            'criteria': 'containing',
                                            'value': 'oxygen',
                                            'format': workbook.add_format({'bg_color': '#FFEB9C'})})

# Boolean columns
# N : ring_system
# O : global_magnetic_field
for column in ('N', 'O'):
    for val, colour in (
        ('TRUE', 'green'),
        ('FALSE', 'red')
    ):
        worksheet1.conditional_format(f'{column}2:{column}10000', {'type': 'text',
                                                    'criteria': 'containing',
                                                    'value': val,
                                                    'format': workbook.add_format({'font_color': colour,
                                                                                'bold': True})})

# N (ring_system), O (global_magnetic_field) are boolean

# P (average_temperature_celsius) is temperature
worksheet1.conditional_format('P2:P10000', {'type': 'data_bar',
                                            'bar_solid': True,
                                            'bar_border_color': 'black',
                                            'bar_negative_color_same': False,
                                            'bar_negative_border_color_same': True,
                                            'bar_color': 'red',
                                            'bar_negative_color': 'blue'})

# put filter at the top of each column
(max_row, max_col) = df.shape
worksheet1.autofilter(0, 0, max_row, max_col-1)

# format column headers
columns = list(df.columns)
for index, col in enumerate(columns):
    worksheet1.write(0, index, col, workbook.add_format({'bg_color': '#951F06',
                                                          'valign': 'vcenter',
                                                          'align': 'center',
                                                          'bold': True,
                                                          'font_color': '#FFFFFF'}))

# ADD COLUMN
# add formula
for i in range(2, 1000):
    worksheet1.write_formula(f'AI{i}', f'=SUM(AG{i}, AH{i})')

# embed image
# NOTE: available in excel 365 versions from 2023 onwards, for older versions a #value! error is displayed
# worksheet1.set_row(1, 60)
# worksheet1.embed_image('C11', 'newplot.png')

# Freeze the top row
worksheet1.freeze_panes(1, 0)

# insert a chart
empty_df = pd.DataFrame()
empty_df.to_excel(writer, sheet_name='plots')
worksheet2 = writer.sheets['plots']

chart1 = workbook.add_chart({'type': 'scatter'})
chart1.add_series({'name': 'chartName',
                  'categories': '=main!$T$2:$T$8',
                  'values': '=main!$W$2:$W$8'})
chart1.set_title({'name': 'Escape velocity and gravity'})
chart1.set_x_axis({'name': 'escape_velocity_km_s'})
chart1.set_y_axis({'name': 'gravity_m_s2'})
chart1.set_style(5)
worksheet2.insert_chart('B2', 
                        chart1)


### Save the file
writer.close()


In [ ]:
df.shape

(8, 34)

## Conditional formatting

In [51]:
import random

random.randint(0, 10)

8

In [54]:
import pandas as pd
import numpy as np

sales = [-10, -15, 0, 1850.00, 430.20, 998.10, 760.00]
df_sales = pd.DataFrame({
    "id": [i for i in range(len(sales))],
    "price_increase": [i * random.randint(5, 10) for i in range(len(sales))],
    "fund_amount": sales,
    'amount_adjusted': [i*11 for i in sales], 
    # "date": pd.date_range("2026-01-01", periods=5)
})
df_sales

,id,price_increase,fund_amount,amount_adjusted
0,0,0,-10.0,-110.0
1,1,7,-15.0,-165.0
2,2,18,0.0,0.0
3,3,21,1850.0,20350.0
4,4,36,430.2,4732.2
5,5,50,998.1,10979.1
6,6,60,760.0,8360.0


In [55]:
with pd.ExcelWriter(f'output/excel_cond_format_df.xlsx', 
                    engine = "xlsxwriter"
                    ) as writer:
    df_sales.to_excel(writer, 
                      index = False, 
                      sheet_name = 'Tab sales'
                      )
    wb  = writer.book
    ws  = writer.sheets['Tab sales']

    # ------------- Helpers -------------
    def colnum_to_excel(n: int) -> str:
        """0-based column index -> Excel letters"""
        s = ""
        n += 1
        while n:
            n, r = divmod(n - 1, 26)
            s = chr(65 + r) + s
        return s



    nrows = len(df_sales)
    ncols = len(df_sales.columns)

    # ------------- Header filter + freeze panes -------------
    ws.autofilter(0, 0, nrows, ncols - 1)
    ws.freeze_panes(1, 0)

    # ------------- Number formats -------------
    thousands_fmt = wb.add_format({"num_format": "#,##0"})
    red_zero_thousands_fmt = wb.add_format({"font_color": "red", "num_format": "#,##0"})

    # Apply thousands separator as the default column format for the two count columns
    for name in ["fund_amount"]:
        idx = df_sales.columns.get_loc(name)
        # Keep column width as set above; attach thousands format as default
        ws.set_column(idx, idx, None, thousands_fmt)

    # If '%_change' is numeric, optionally display as percent if values look like decimals (0.12 -> 12%)
    pct_idx = df_sales.columns.get_loc("amount_adjusted")
    if pd.api.types.is_numeric_dtype(df_sales["amount_adjusted"]):
        s = df_sales["amount_adjusted"].dropna()
        if len(s) and (s.between(-1.5, 1.5)).mean() > 0.8:
            pct_display_fmt = wb.add_format({"num_format": "0.00%"})
        else:
            # Already in percentage points (e.g., 12.3 meaning 12.3%)
            pct_display_fmt = wb.add_format({"num_format": "0.00"})
        ws.set_column(pct_idx, pct_idx, None, pct_display_fmt)

    # ------------- Conditional formatting: zero values in count columns -------------
    for name in ["price_increase"]:
        col_idx = df_sales.columns.get_loc(name)
        col_letter = colnum_to_excel(col_idx)
        cell_range = f"{col_letter}2:{col_letter}{nrows + 1}"  # data rows
        ws.conditional_format(cell_range, {
            "type": "cell",
            "criteria": "==",
            "value": 0,
            "format": red_zero_thousands_fmt  # preserves thousands separator and turns text red
        })

    # ------------- Conditional formatting: %_change red↔white↔green -------------
    for name in ['fund_amount']:
        col_idx = df_sales.columns.get_loc(name)
        pct_letter = colnum_to_excel(col_idx)
        pct_range = f"{pct_letter}2:{pct_letter}{nrows + 1}"

        # Center the gradient at 0 (white). Negatives shade to red, positives shade to green.
        ws.conditional_format(pct_range, {
            "type": "3_color_scale",
            "min_color": "#F8696B",  # red
            "mid_color": "#FFFFFF",  # white
            "max_color": "#63BE7B",  # green
            "mid_type": "num",
            "mid_value": 0
        })

    # ------------- Auto-fit column widths -------------
    # We estimate width from the longest string among header and data.
    # For numeric columns that will be shown with thousand separators,
    # we format as such before measuring so the width accounts for commas.
    max_width_cap = 600
    for col_idx, col_name in enumerate(df_sales.columns):
        series = df_sales[col_name]
        if pd.api.types.is_numeric_dtype(series) and col_name in (
            "fund_amount", 'amount_adjusted'
        ):
            # Represent with thousands separators for width estimate
            series_as_str = series.map(lambda x: f"{int(x):,}" if pd.notna(x) else "")
        else:
            series_as_str = series.astype(str)
        max_len_data = series_as_str.map(len).max() if len(series_as_str) else 0
        header_len   = len(str(col_name))
        if col_name == 'amount_adjusted':
            print(series_as_str, col_name, len(str(col_name)), header_len, max_len_data)
        best_width   = min(
            max(
                max_len_data, 
                header_len
            ) + 10, 
            max_width_cap
        )
        ws.set_column(col_idx, col_idx, best_width)


0      -110
1      -165
2         0
3    20,350
4     4,732
5    10,979
6     8,360
Name: amount_adjusted, dtype: object amount_adjusted 15 15 6


## Save multiple dfs to excel

In [56]:
import random 

import numpy as np
import pandas as pd
from openpyxl.styles import Font, Alignment


In [60]:
# DataFrame 1: Employees
df_employees = pd.DataFrame({
    "employee_id": [1, 2, 3, 4],
    "name": ["Alice Catniss", "Bobra K.", "Carlos N.J.", "Dranei B."],
    "department": ["Data Science", "Finance", "Marketing", "Data Science"],
    "salary_usd": [90000, 75000, 68000, 95000]
})

# DataFrame 2: Sales Data
df_sales = pd.DataFrame({
    "sale_id": np.arange(1, 6),
    "employee_id": [1, 1, 2, 4, 3],
    "amount": [1200.50, 850.00, 430.20, 998.10, 760.00],
    "date": pd.date_range("2026-01-01", periods=5)
})

# Dataframe 3
ids = [1, 2, 3, 4, 5]
df_fake = pd.DataFrame({
    'id': ids,
    'variance': [i * 100 for i in ids],
    'occlusion': [i * random.randint(1, 10) for i in ids]
})
df_employees

,employee_id,name,department,salary_usd
0,1,Alice Catniss,Data Science,90000
1,2,Bobra K.,Finance,75000
2,3,Carlos N.J.,Marketing,68000
3,4,Dranei B.,Data Science,95000


In [77]:
PATH = "output/df_excel_comparison.xlsx"
SHEET_NAME = 'Compare'
LEFT_OFFSET_COL = 3

def colnum_to_excel(n: int) -> str:
    """0-based column index -> Excel letters"""
    s = ""
    n += 1
    while n:
        n, r = divmod(n - 1, 26)
        s = chr(65 + r) + s
    return s

with pd.ExcelWriter(PATH, 
                    engine="openpyxl"
                    ) as writer:
    # Left table
    df_employees.to_excel(writer, 
                          sheet_name = SHEET_NAME, 
                          startrow = 2, 
                          startcol = LEFT_OFFSET_COL, 
                          index = False
                          )
    # Initialise worksheet object with `writer.book`
    ws = writer.book[SHEET_NAME]
    # main
    cell = ws.cell(row = 2, column = LEFT_OFFSET_COL + 1)
    cell.value = 'Employees dataframe'
    cell.font = Font(bold=True)
    for col_idx, col_name in enumerate(df_employees.columns):
        series = df_employees[col_name]
        if pd.api.types.is_numeric_dtype(series):
            series_as_str = series.map(lambda x: f"{int(x):,}" if pd.notna(x) else "")
        else:
            series_as_str = series.astype(str)
        max_len_data = series_as_str.map(len).max() if len(series_as_str) else 0
        header_len = len(str(col_name))
        best_width   = min(
            max(
                max_len_data, 
                header_len
            ) + 10, 
            max_width_cap
            )
        idxx = colnum_to_excel(
            col_idx + LEFT_OFFSET_COL
        )
        # this is openpyxl's alternative to ws.set_column
        ws.column_dimensions[idxx].width = best_width

    # Middle table (offset by df1’s width + 2 spacer columns)
    left_width = LEFT_OFFSET_COL + df_employees.shape[1] + 2
    df_sales.to_excel(writer, 
                      sheet_name = SHEET_NAME, 
                      startrow = 2, 
                      startcol = left_width, 
                      index = False
                      )
    cell = ws.cell(row = 2, column = left_width + 1)
    cell.value = 'Sales dataframe'
    cell.font = Font(bold = True)
    for col_idx, col_name in enumerate(df_sales.columns):
        series = df_sales[col_name]
        if pd.api.types.is_numeric_dtype(series):
            series_as_str = series.map(lambda x: f"{int(x):,}" if pd.notna(x) else "")
        else:
            series_as_str = series.astype(str)
        max_len_data = series_as_str.map(len).max() if len(series_as_str) else 0
        header_len = len(str(col_name))
        best_width   = min(
            max(
                max_len_data, 
                header_len
            ) + 10, 
            max_width_cap
            )
        idxx = colnum_to_excel(
            col_idx + LEFT_OFFSET_COL + len(df_employees.columns.tolist()) + 2
        )
        # this is openpyxl's alternative to ws.set_column
        ws.column_dimensions[idxx].width = best_width

    # Right table (offset by df1 + spacer + df2’s width + 2 spacers)
    middle_width = df_sales.shape[1] + 2
    df_fake.to_excel(writer, 
                     sheet_name = SHEET_NAME, 
                     startrow = 2, 
                     startcol = left_width + middle_width, 
                     index = False
                     )
    cell = ws.cell(row = 2, column = left_width + middle_width + 1)
    cell.value = 'Difference'
    cell.font = Font(bold = True)

In [84]:
os.getcwd()

'c:\\Users\\Evgeni Zorin\\OneDrive - Euromonitor International\\Desktop\\Handbook_Data-Science'

0